# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [2]:
import pandas as pd
import seaborn as sns
from seaborn import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib inline

In [3]:
df = pd.read_csv('previsao_de_renda.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [5]:
df.drop('Unnamed: 0', axis='columns', inplace=True)

In [6]:
df = df.fillna(0)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               15000 non-null  object 
 1   id_cliente             15000 non-null  int64  
 2   sexo                   15000 non-null  object 
 3   posse_de_veiculo       15000 non-null  bool   
 4   posse_de_imovel        15000 non-null  bool   
 5   qtd_filhos             15000 non-null  int64  
 6   tipo_renda             15000 non-null  object 
 7   educacao               15000 non-null  object 
 8   estado_civil           15000 non-null  object 
 9   tipo_residencia        15000 non-null  object 
 10  idade                  15000 non-null  int64  
 11  tempo_emprego          15000 non-null  float64
 12  qt_pessoas_residencia  15000 non-null  float64
 13  renda                  15000 non-null  float64
dtypes: bool(2), float64(3), int64(3), object(6)
memory usa

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.

2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

In [76]:
df['tipo_residencia'].value_counts()

tipo_residencia
8.594606     6
7.148377     6
9.192819     6
10.678996    6
6.591619     6
            ..
9.035370     1
8.965345     1
8.655559     1
8.648202     1
7.057028     1
Name: count, Length: 9786, dtype: int64

In [75]:
df['tipo_residencia'] = np.log(df['renda'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   data_ref               15000 non-null  object 
 1   id_cliente             15000 non-null  int64  
 2   sexo                   15000 non-null  object 
 3   posse_de_veiculo       15000 non-null  bool   
 4   posse_de_imovel        15000 non-null  bool   
 5   qtd_filhos             15000 non-null  int64  
 6   tipo_renda             15000 non-null  object 
 7   educacao               15000 non-null  object 
 8   estado_civil           15000 non-null  object 
 9   tipo_residencia        15000 non-null  float64
 10  idade                  15000 non-null  int64  
 11  tempo_emprego          15000 non-null  float64
 12  qt_pessoas_residencia  15000 non-null  float64
 13  renda                  15000 non-null  float64
 14  log_renda              15000 non-null  float64
dtypes:

In [77]:
y,x = patsy.dmatrices('log_renda ~ + sexo + tipo_renda + C(educacao, Treatment(2)) + estado_civil', data=df)
reg = sm.OLS(y, x).fit().summary()
reg

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_renda   R-squared:                       0.157
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     215.3
Date:                Wed, 24 Jul 2024   Prob (F-statistic):               0.00
Time:                        15:06:45   Log-Likelihood:                -18141.
No. Observations:               15000   AIC:                         3.631e+04
Df Residuals:                   14986   BIC:                         3.642e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            7.9488      0.013    613.056      0.000       7.923       7.974
sexo[T.M]                                            0.6936      0.015     47.510      0.000       0.665       0.722
tipo_renda[T.Bolsista]                               0.4110      0.271      1.517      0.129      -0.120       0.942
tipo_renda[T.Empresário]                             0.0729      0.017      4.377      0.000       0.040       0.106
tipo_renda[T.Pensionista]                           -0.1413      0.019     -7.344      0.000      -0.179      -0.104
tipo_renda[T.Servidor público]                       0.2197      0.025      8.870      0.000       0.171       0.268
C(educacao, Treatment(2))[T.Primário]                0.0105      0.064      0.164      0.870      -0.115       0.136
C(educacao, Treatment(2))[T.Pós graduação]          -0.0620      0.160     -0.389      0.697      -0.375       0.251
C(educacao, Treatment(2))[T.Superior completo]       0.0940      0.014      6.594      0.000       0.066       0.122
C(educacao, Treatment(2))[T.Superior incompleto]    -0.1381      0.035     -3.946      0.000      -0.207      -0.069
estado_civil[T.Separado]                             0.0410      0.029      1.431      0.152      -0.015       0.097
estado_civil[T.Solteiro]                            -0.0439      0.021     -2.110      0.035      -0.085      -0.003
estado_civil[T.União]                               -0.0725      0.026     -2.788      0.005      -0.123      -0.022
estado_civil[T.Viúvo]                                0.0462      0.033      1.421      0.155      -0.018       0.110
==============================================================================
Omnibus:                      153.271   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              175.992
Skew:                           0.197   Prob(JB):                     6.08e-39
Kurtosis:                       3.355   Cond. No.                         48.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# avaliação de paramentros: 
    - Observando os parametros é possivel dizer que não seria um modelo muito bom para se utilizar.
    - O P-value da variavel "educacao" por exemplo, é a que apresenta mais categorias acima de 0,05. E por esse motivo a melhor escolha a se tomar para melhorar o modelo seria retirar essa variavel.

# 2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.


In [80]:
y,x = patsy.dmatrices('log_renda ~ tipo_renda + C(educacao, Treatment(2))', data=df)
reg = sm.OLS(y, x).fit().summary()
reg

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_renda   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     51.67
Date:                Wed, 24 Jul 2024   Prob (F-statistic):           3.87e-83
Time:                        15:10:20   Log-Likelihood:                -19221.
No. Observations:               15000   AIC:                         3.846e+04
Df Residuals:                   14991   BIC:                         3.853e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            8.2138      0.011    714.800      0.000       8.191       8.236
tipo_renda[T.Bolsista]                               0.1559      0.291      0.536      0.592      -0.414       0.726
tipo_renda[T.Empresário]                             0.0366      0.018      2.052      0.040       0.002       0.072
tipo_renda[T.Pensionista]                           -0.3080      0.020    -15.429      0.000      -0.347      -0.269
tipo_renda[T.Servidor público]                       0.1533      0.027      5.771      0.000       0.101       0.205
C(educacao, Treatment(2))[T.Primário]                0.0398      0.069      0.581      0.562      -0.095       0.174
C(educacao, Treatment(2))[T.Pós graduação]          -0.1326      0.171     -0.774      0.439      -0.468       0.203
C(educacao, Treatment(2))[T.Superior completo]       0.0841      0.015      5.499      0.000       0.054       0.114
C(educacao, Treatment(2))[T.Superior incompleto]    -0.1109      0.038     -2.953      0.003      -0.185      -0.037
==============================================================================
Omnibus:                      263.615   Durbin-Watson:                   2.030
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              297.920
Skew:                           0.286   Prob(JB):                     2.03e-65
Kurtosis:                       3.387   Cond. No.                         45.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# testes
    - A partir da célula acima, fiz testes de 1 a 1 com cada variavel junto da variavel resposta.
    - usei como referencia o valor do p-value. As que fossem menores que 0,05 eu coloquei no modelo final e fiz mais testes.

In [79]:
y,x = patsy.dmatrices('log_renda ~  C(educacao, Treatment(2))', data=df)
reg = sm.OLS(y, x).fit().summary()
reg

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_renda   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     18.07
Date:                Wed, 24 Jul 2024   Prob (F-statistic):           8.16e-15
Time:                        15:09:43   Log-Likelihood:                -19389.
No. Observations:               15000   AIC:                         3.879e+04
Df Residuals:                   14995   BIC:                         3.883e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
====================================================================================================================
                                                       coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------
Intercept                                            8.1673      0.009    873.899      0.000       8.149       8.186
C(educacao, Treatment(2))[T.Primário]               -0.0210      0.069     -0.303      0.762      -0.157       0.115
C(educacao, Treatment(2))[T.Pós graduação]          -0.0663      0.173     -0.383      0.702      -0.406       0.273
C(educacao, Treatment(2))[T.Superior completo]       0.1220      0.015      7.994      0.000       0.092       0.152
C(educacao, Treatment(2))[T.Superior incompleto]    -0.0542      0.038     -1.434      0.152      -0.128       0.020
==============================================================================
Omnibus:                      332.346   Durbin-Watson:                   2.030
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              376.220
Skew:                           0.331   Prob(JB):                     2.02e-82
Kurtosis:                       3.406   Cond. No.                         25.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
y,x = patsy.dmatrices('log_renda ~  estado_civil', data=df)
reg = sm.OLS(y, x).fit().summary()
reg

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_renda   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     23.61
Date:                Wed, 24 Jul 2024   Prob (F-statistic):           1.72e-19
Time:                        15:01:45   Log-Likelihood:                -19378.
No. Observations:               15000   AIC:                         3.877e+04
Df Residuals:                   14995   BIC:                         3.880e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept                    8.2471      0.009    961.003      0.000       8.230       8.264
estado_civil[T.Separado]    -0.0778      0.031     -2.515      0.012      -0.138      -0.017
estado_civil[T.Solteiro]    -0.0991      0.022     -4.411      0.000      -0.143      -0.055
estado_civil[T.União]       -0.1268      0.028     -4.500      0.000      -0.182      -0.072
estado_civil[T.Viúvo]       -0.2795      0.034     -8.189      0.000      -0.346      -0.213
==============================================================================
Omnibus:                      303.933   Durbin-Watson:                   2.031
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              341.534
Skew:                           0.315   Prob(JB):                     6.87e-75
Kurtosis:                       3.386   Cond. No.                         4.96
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [81]:
y,x = patsy.dmatrices('log_renda ~ sexo + tipo_renda + estado_civil ', data=df)
reg = sm.OLS(y, x).fit().summary()
reg


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_renda   R-squared:                       0.153
Model:                            OLS   Adj. R-squared:                  0.153
Method:                 Least Squares   F-statistic:                     301.9
Date:                Wed, 24 Jul 2024   Prob (F-statistic):               0.00
Time:                        15:10:43   Log-Likelihood:                -18175.
No. Observations:               15000   AIC:                         3.637e+04
Df Residuals:                   14990   BIC:                         3.645e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          7.9753      0.012    673.056      0.000       7.952       7.998
sexo[T.M]                          0.6906      0.015     47.219      0.000       0.662       0.719
tipo_renda[T.Bolsista]             0.4785      0.271      1.764      0.078      -0.053       1.010
tipo_renda[T.Empresário]           0.0806      0.017      4.848      0.000       0.048       0.113
tipo_renda[T.Pensionista]         -0.1437      0.019     -7.487      0.000      -0.181      -0.106
tipo_renda[T.Servidor público]     0.2373      0.025      9.606      0.000       0.189       0.286
estado_civil[T.Separado]           0.0461      0.029      1.609      0.108      -0.010       0.102
estado_civil[T.Solteiro]          -0.0435      0.021     -2.090      0.037      -0.084      -0.003
estado_civil[T.União]             -0.0770      0.026     -2.958      0.003      -0.128      -0.026
estado_civil[T.Viúvo]              0.0393      0.033      1.208      0.227      -0.024       0.103
==============================================================================
Omnibus:                      152.762   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              175.174
Skew:                           0.197   Prob(JB):                     9.15e-39
Kurtosis:                       3.353   Cond. No.                         45.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.

# Conclusão
    • Para finalizar e ter o melhor modelo, utilizei as variáveis qualitativas "sexo", "tipo_renda" e "estado_civil". Com essas variaveis obtive um R-squared melhor que o primeiro modelo. Na minha opinião o ultimo modelo seria a melhor alternativa para ser usado.